# 1. MySQL数据库字符集介绍

简单的说是一套文字符号及其编码、比较规则的集合。

MySQL数据库字符集包括
- 字符集（CHARACTER）
- 校对规则（COLLATION）两个概念。

其中，字符集是用来定义MySQL数据字符串的存储方式，而校对规则则是定义比较字符串的方式。

前面建库的语句中CHARACTER SET latin1即为数据库字符集而COLLATE latin1_swedish_ci 为校对字符集，有关字符集详细内容参考mysql手册，第10张字符集章节。

# 2. MySQL数据库常见字符集介绍

使用MySQL时常用的字符集有四种：
- utf8，长度3，中英文混合的环境，建议使用此字符集
- gbk，长度2，不是国际标准
- latin1，长度1，MySQL的默认字符集
- utf8mb4，长度4，UTF-8 Unicode，移动互联网。

# 3. MySQL如何选择合适的字符集
1. 如果处理各种各样的文字，发布到不同国家和地区，应选Unicode字符集。对mysql来说就是UTF-8（每个汉字三个字节），如果应用需处理英文，有少量汉字使用UTF-8字符集更好。

2. 如果只需支持中文，并且数据量很大，性能要求也很高，可选GBK（定长，每个汉字占双字节，英文也占双字节），处理大量运算，比较顺序等。定长字符集更快，性能高。

3. 处理移动互联网业务，可能需要使用utf8mb4字符集。

# 4. 查看当前MySQL系统支持的字符集

# 5. mysql数据库默认设置的字符集是什么？

查看mysql当前的字符集设置情况:

提示：   
默认情况下character_set_client，character_set_connection，character_set_results三者的字符集和系统的字符集是一致的，是同时修改的。


更改linux系统字符集变量后(/etc/sysconfig/i18n)，查看MySQL中字符集的变化。

我们发现character_set_connection，character_set_client，character_set_server 三者的字符集和系统的一致也都改成utf8了。

注意：虚拟机上安装的MySQL在二进制编译时就指定了默认的字符集，可以通过/etc/my.cnf更改字符集，但是更改系统字符集/etc/sysconfig/i18n，并不能实现上述跟随系统字符集变更的状况。

更加稳定。

# 6. 执行set names latin1 到底做了什么？

无论linux系统的字符集是gb2312还是utf8，默认情况下插入数据都是乱码的。

set names latin1 ，将character_set_client, character_set_connection, character_set_results的参数修改为latin1。

# 7. mysql命令参数--default-character-set=latin1在做什么？

--default-character-set=latin1，修改了character_set_client, character_set_connection, character_set_results的参数修改为latin1

# 8. 确保MySQL数据库插入数据不乱码解决方案

## 8.1 统一客户端字符集
1. MySQL数据库的下面几个字符集（客户端和服务端）统一成一个字符集才能确保插入的中文数据库可以正常输出。当然，linux系统的字符集也要尽可能和数据库字符集统一。

2. mysql数据库字符集的含义：

其中，①②④三个参数默认情况采用linux系统字符集设置，人工登录数据库执行set names latin1以及mysql指定字符集登录操作，都是改变mysql客户端的client、connection、results3个参数的字符集都为latin1,从而解决插入乱码问题，这个操作可以在my.cnf配置文件里修改mysql客户端的字符集，配置方法如下：

3. 修改完客户端字符集不用set查询表数据就不会乱码了

## 8.2 统一MySQL服务端字符集

提示：以上在[mysqld]下设置的参数会更改下面2个参数的字符集设置。

| Variable_name            | Value|
|::|::|
| character_set_database   | utf8 |
| character_set_server     | utf8 |

这个时候我们再修改系统字符集(/etc/sysconfig/i18n)，mysql数据库字符集就不乱码了.

## 8.3 统一字符集方法总结

保证数据库数据不乱码的方法：建议中英文环境选择utf8，linux系统，客户端，服务端，库，表，程序字符集统一。

1. Linux系统字符集统一utf8     
    /etc/sysconfig/i18n 

     提示，Linux客户端也要更改字符集，例如：xshell，secureCRT

2. 临时修改：set names latin1
3. 永久修改：更改my.cnf配置文件   
                [client]
                Default-character-set=latin1
                [client]
                Default-character-set=latin1
4. 库表，程序制定字符集建库     
create database linzhongniao_utf8 DEFAULT CHARACTER SET UTF8 COLLATE  utf8_general_ci;

# 9. 如何更改生产MySQL数据库表的字符集

对于已有数据库，想修改字符集不能直接通过“alter database character set ”或者”alter table tablename character set ”，这两个命令都不能更新已有数据的字符集。   
而只是对新创建的表或者数据生效。

**已经有记录的字符集的调整必须将数据导出，经过修改字符集之后重新导入才可完成。**

### 修改数据库默认编码命令：  

alter database [your db name] charset [your character setting]


下面模拟将latin1字符集的数据库修改成GBK字符集的过程。

#### 1. 导出表结构

使用mysqldump的-d参数导出表结构

> mysqldump –uroot –p123456 –-default-character-set=latin1 –d dbname>alltable.sql –-default-character-set=gbk 

- 表示以GBK字符集进行连接;
- –d只导表结构。


#### 2. 然后编辑alltable.sql将latin1用sed替换成GBK

#### 3. 确保数据不在更新，导出所有数据

> mysqldmup –uroot –p123456 –-quick –-no-create-info –-extended-insert –-default-character-set=latin1 dbname>alltables.sql


参数说明：
- --quick：用于转储大的表，强制mysqldump从服务器一次一行的检索数据而不是检索所有行并输出前CACHE到内存中。
- --no-create-info：不创建CREATE TABLE 语句。
- --extended-insert：使用包括几个VALUES列表的多行INSERT语法，这样文件更小节省IO导入数据非常快。
- --default-character-set=latin1：按照原有字符集导出数据，这样导出的文件中，所有中文都是可见的，不会保存成乱码。

#### 4. 打开alltable.sql将set names latin1修改成set names gbk(或者修改my.cnf配置文件)

#### 5. 建库
> create database dbname default charset gbk;
#### 6.创建表执行，alltable.sql
> mysql –uroot –p123456 dbname<alltable.sql

#### 7. 导入数据
> mysql –uroot –p123456 dbname<alltables.sql